In [1]:
import os
os.environ['OMP_NUM_THREADS']='2'
os.environ['LD_LIBRARY_PATH']=''
os.environ['CUDA_LAUNCH_BLOCKING']='1'

In [2]:
%cd /home/pz281@ad.eng.cam.ac.uk/mnt/PhD/Pro_Down_SR

/home/pz281@ad.eng.cam.ac.uk/mnt/PhD/Pro_Down_SR


In [3]:
from data_generation import *
from scipy.linalg import sqrtm
from downscaling import *
from utils import *
import random

/home/pz281@ad.eng.cam.ac.uk/anaconda3/envs/torch/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Langevin & Training Downscale Network

### Upscale By 4

In [4]:
N_low = 20
N_high = 80
scale = 4

h_low = 1/(N_low-1)
x_low = np.arange(0,1.0001,h_low)
y_low = np.arange(0,1.0001,h_low)

h_high = 1/(N_high-1)
x_high = np.arange(0,1.0001,h_high)
y_high = np.arange(0,1.0001,h_high)

In [5]:
A_high = create_A(N_high)
A_low = create_A(N_low)

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [7]:
sigma = 0.002
G = np.eye(N_high**2) * sigma**2
G_inverse = np.eye(N_high**2) * (1/sigma**2)
'''
G = gaussian_kernal(x_high,y_high,l,sigma,N_high)
G_inverse = np.linalg.inv(G+1e-6*np.eye(N_high**2))'''

'\nG = gaussian_kernal(x_high,y_high,l,sigma,N_high)\nG_inverse = np.linalg.inv(G+1e-6*np.eye(N_high**2))'

In [8]:
# Turn matrices to tensors
G = torch.tensor(G).to(torch.float32).to(device)
G_inverse = torch.tensor(G_inverse).to(torch.float32).to(device)
A_high = torch.tensor(create_A(N_high)).to(torch.float32).to(device)

In [9]:
# Store sparse matrices as sparse tensor
A_high = A_high.to_sparse()
G = G.to_sparse()
G_inverse = G_inverse.to_sparse()
operator = torch.spmm(A_high.T,G_inverse).to(device)

In [11]:
dataset = DataFromH5File4("data/20_80_low_forcing.h5")

trainset = random.sample(range(0, 128), 100)
testset = [i for i in range(0,128) if i not in trainset]

In [12]:
def sample_data():
    coefficient = random.sample(trainset,1)[0]
    forcing = dataset[coefficient][0]
    lr = dataset[coefficient][1]
    
    return forcing, lr


def sample_p_0():
    # Randomly sampling for initialisation of the Langevin dynamics
    # prior = torch.randn(*[batch_size,1,20,20]).to(device)
    
    # Set the u_low_mean to the initialisation of the Langevin dynamics
    posterior_initial = torch.randn([N_high,N_high]).to(torch.float32)
    posterior_initial = torch.tensor(posterior_initial).to(device).to(torch.float32)
    
    return posterior_initial

    
def ula_posterior_preconditioner(z, b_high, x, G):
    """
    Langevin dynamics with preconditioner
    """
    z = z.clone().detach().requires_grad_(True)
    for i in range(K):
        # Grad log-likelihood
        x_hat = G(z.reshape(1,N_high,N_high)).reshape(N_low,N_low)
        log_likelihood = (-1/(2*math.pow(ll_sigma, 2)) * torch.matmul((x-x_hat).reshape(1,N_low**2),(x-x_hat).reshape(N_low**2,1)))
        grad_ll = torch.autograd.grad(log_likelihood, z)[0]

        # Grad prior
        difference = torch.spmm(A_high,z.reshape(N_high*N_high,1)) - b_high.reshape(N_high**2,1)
        # log_prior = - 0.5 * difference.T @ G_inverse @ difference
        # grad_log_prior = torch.autograd.grad(log_prior, z)[0]
        grad_log_prior = (- torch.spmm(operator,difference)).reshape(N_high,N_high)
        
        # Random noise term
        W = torch.randn(*[N_high,N_high]).to(device)
        # random = torch.matmul(G_sqrt,W.reshape(N_high**2,1)).reshape(N_high,N_high)
        
        z = z + 0.5 * s ** 2 * grad_log_prior + 0.5 * s ** 2 * grad_ll + s * W
        # chains_evolution.append(z.cpu().data.numpy())   
           
    return z.detach()

In [1]:
# Train with sampled data
epoch_num = 1000
lr = 0.003
gamma = 0.5
K = 800
s = 0.0004
ll_sigma = 0.001
step_size = 20
minimum_loss = float('inf')
loss_track = []

G = DownScale()
G.apply(weights_init_xavier).to(device)
mse = nn.MSELoss(reduction='sum')
optG = torch.optim.Adam(G.parameters(), lr = lr, weight_decay=0, betas=(0.5, 0.999))
r_scheduleG = torch.optim.lr_scheduler.StepLR(optG, step_size=step_size, gamma=gamma)

# Logger info
dir_name = f'models/model1_bicubic/lr{lr}_gamma{gamma}_stepsize{step_size}_K{K}'
makedir(dir_name)
logger = setup_logging('job0', dir_name, console=True)
logger.info(f'Training for {epoch_num} epoches and learning rate is {lr}')

for epoch in range(1, epoch_num+1):
    
    b_high, low_res = sample_data()
    b_high = torch.tensor(b_high).to(torch.float32).to(device)
    low_res = torch.tensor(low_res).to(torch.float32).to(device)
    
    posterior_initial = sample_p_0()
    posterior_final = ula_posterior_preconditioner(posterior_initial, b_high, low_res, G)

    optG.zero_grad()
    
    out = G(posterior_final.reshape(1,N_high,N_high)).reshape(N_low,N_low)
    loss = mse(out,low_res)
        
    '''observation = torch.spmm(H,posterior_final.reshape(N_high*N_high,1)).reshape(1,N_low,N_low)
    out = observation + G(observation)
    loss = mse(out,low_res.reshape(1,N_low,N_low))'''
    loss.backward()
    optG.step()
    
    if loss < minimum_loss:
        save_model(dir_name, epoch, 'best_model', r_scheduleG, G, optG)
        minimum_loss = loss
            
    if epoch%100 == 0:
        save_model(dir_name, epoch, 'model_epoch_{}'.format(epoch), r_scheduleG, G, optG)
    
    save_model(dir_name, epoch, 'current_epoch', r_scheduleG, G, optG)
    loss_track.append(loss.cpu().data.numpy())
    np.save(f'{dir_name}/chains/loss_curve.npy', np.array(loss_track))
    
    print("Epoch:", epoch, "Loss:", loss)

    r_scheduleG.step()

NameError: name 'DownScale' is not defined